## Predictive Modelling ML Project PythonTemplate

## Problem Definition
For this project we will investigate the Boston House Price dataset. Each record in the database
describes a Boston suburb or town. The data was drawn from the Boston Standard Metropolitan
Statistical Area (SMSA) in 1970. The attributes are defined as follows (taken from the UCI
Machine Learning Repository1):https://www.kaggle.com/vikrishnan/boston-house-prices

1. CRIM: per capita crime rate by town
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS: proportion of non-retail business acres per town
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX: nitric oxides concentration (parts per 10 million)
6. RM: average number of rooms per dwelling
7. AGE: proportion of owner-occupied units built prior to 1940
8. DIS: weighted distances to five Boston employment centers
9. RAD: index of accessibility to radial highways
10. TAX: full-value property-tax rate per USD 10,000
11. PTRATIO: pupil-teacher ratio by town
12. [B:1000(Bk − 0:63)2] where Bk is the proportion of blacks by town
13. LSTAT: % lower status of the population
14. MEDV: Median value of owner-occupied homes in $1000s

The input attributes have a mixture of attributes

# 1. PREPARE PROBLEM

## 1.1 Load libraries

In [ ]:
import numpy
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

## 1.2 Load dataset

In [ ]:
filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
'B', 'LSTAT', 'MEDV']
dataset = read_csv(filename, delim_whitespace=True, names=names)

# 2. SUMMARIZE DATA

## 2.1 Descriptive statistics

In [ ]:
# confirm dimensions of dataset
print(dataset.shape)
# We have 506 instances to work with and can confirm the data has 14 attributes including the output attribute MEDV

In [ ]:
# look at data types of each attribute
print(dataset.dtypes)


In [ ]:
# look at first 20 rows of data
print(dataset.head(20))

In [ ]:
# Let’s summarize the distribution of each attribute
set_option('precision', 1)
print(dataset.describe())

In [ ]:
# take a look at the correlation between all of the numeric attributes.
set_option('precision', 2)
print(dataset.corr(method='pearson'))

## 2.2 Data visualizations

It is often useful to look at your data using multiple different visualizations in order to spark ideas. 
- histograms of each attribute helps get a sense of data distributions

In [ ]:
# Unimodal data visualization using histogram. s
dataset.hist(sharex=False, sharey=False, xlabelsize=1, ylabelsize=1)
pyplot.show()

In [ ]:
#look at the same distributions using density plots that smooth them out a bit
dataset.plot(kind='density', subplots=True, layout=(4,4), sharex=False, legend=False,fontsize=1)
pyplot.show()

In [ ]:
# Mulitmodal data visualization using a scatter plot matrix to show interactions between variables
scatter_matrix(dataset)
pyplot.show()

In [ ]:
# correlations between the attributes using a correlations matrix
fig = pyplot.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(dataset.corr(), vmin=-1, vmax=1, interpolation='none')
fig.colorbar(cax)
ticks = numpy.arange(0,14,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(names)
ax.set_yticklabels(names)
pyplot.show()

## 2.3 Conclusions from Visualization exercise
There is a lot of structure in this dataset. We need to think about transforms that we could use
later to better expose the structure which in turn may improve modeling accuracy. So far it
would be worth trying:
- Feature selection and removing the most correlated attributes.
- Normalizing the dataset to reduce the effect of differing scales.
- Standardizing the dataset to reduce the effects of differing distributions.
- binning (discretization)of the data (with more time). This can often improve accuracy for decision tree algorithms.

In [ ]:
# 3. PREPARE DATA
# a) Data cleaning
# b) Feature selection
# c) Data transforms (see below section 4.4)

# 4. EVALUATE ALGORITHMS: Baseline

## 4.1 Split-out validation dataset

In [ ]:
# Split-out validation dataset
array = dataset.values
X = array[:,0:13]
Y = array[:,13]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,
test_size=validation_size, random_state=seed)

## 4.2 Test options and evaluation metric

We will design our test harness. We will use 10-fold cross validation. The dataset is not too small and this is a good standard test harness configuration. 

We will evaluate algorithms using the Mean Squared Error (MSE) metric. MSE will give a gross idea of how wrong all predictions are (0 is perfect).

<b> Mean squared error </b>
Imagine for a regression problem we have the line of best fit and we want to measure the
distance of each point from the regression line. Mean squared error (MSE) is the statistical
measure that would compute these deviations. MSE computes errors by finding the mean
of the squares for each such deviations

In [ ]:
# Test options and evaluation metric
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

## 4.3 Spot check algorithm
Let’s create a baseline of performance on this problem and spot-check a number of different algorithms. We will select a suite of different algorithms capable of working on this regression problem. The algorithms all use default tuning parameters. The six algorithms selected include:

- Linear Algorithms: Linear Regression (LR), Lasso Regression (LASSO) and ElasticNet (EN).
- Nonlinear Algorithms: Classification and Regression Trees (CART), Support Vector Regression (SVR) and k-Nearest Neighbors (KNN).


In [ ]:
# Spot-Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))

## 4.4 Compare algorithms

We will display the mean and standard deviation of MSE for each algorithm as we calculate it and collect the results for use later.

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

### Output
It looks like LR has the lowest MSE, followed closely by CART.

    LR: -21.379856 (9.414264)
    LASSO: -26.423561 (11.651110)
    EN: -27.502259 (12.305022)
    KNN: -41.896488 (13.901688)
    CART: -26.909036 (11.579785)
    SVR: -67.827886 (29.049138)



# 3.3 Data Transformation
The differing scales of the raw data is probably negatively impacting the skill of all of the algorithms and
perhaps more so for SVR and KNN. In this section we will look at standardizing the data then later run the same
algorithms on a standardized copy of the data.

This is where the data is transformed such that each attribute has a mean value of
zero and a standard deviation of 1. We also need to avoid data leakage when we transform the
data. A good way to avoid leakage is to use pipelines that standardize the data and build the
model for each fold in the cross validation test harness. That way we can get a fair estimation
of how each model with standardized data might perform on unseen data.


In [ ]:
# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO',Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN',ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART',DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))

In [ ]:
# evaluate each algorithm on a standardized dataset
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# 5. IMPROVE ACCURACY WITH TUNING

## 5.1 Algorithm tuning

In [ ]:
# KNN Algorithm tuning
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
k_values = numpy.array([1,3,5,7,9,11,13,15,17,19,21])
param_grid = dict(n_neighbors=k_values)
model = KNeighborsRegressor()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)

#display the mean and standard deviation scores as well as the best performing value for k
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## 5.2 Ensembles Methods

In [ ]:
# ensembles
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB',AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM',GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF',RandomForestRegressor())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET',ExtraTreesRegressor())])))

In [ ]:
# evaluate each algorithm on the same test harness as before
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

#### 5.2.1 Tune Ensemble Methods
The default number of boosting stages to perform (n estimators) is 100. This is a good
candidate parameter of Extra Trees to tune. Often, the larger the number of boosting
stages, the better the performance but the longer the training time. 

In this section we look at tuning the number of stages for Extra Trees. Below we define a parameter grid
n estimators values from 50 to 400 in increments of 50. Each setting is evaluated using 10-fold
cross validation.


In [ ]:
# Tune GBM on scaled dataset
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400]))
model = ExtraTreesRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)

# As before, we can summarize the best configuration and get an idea of how performance changed with each different configuration.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# 6. FINALIZE MODEL
In this section we will finalize the Extra Trees model and evaluate it on our hold out validation dataset. 

## 6.1 Predictions on validation dataset 
First we need to prepare the model and train it on the entire training dataset. This includes standardizing the training dataset before training.

In [ ]:
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = ExtraTreesRegressor(random_state=seed, n_estimators=200)
model.fit(rescaledX, Y_train)

In [ ]:
# We then scale the inputs for the validation dataset and generate predictions.
# transform the validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(Y_validation, predictions))

### Output
The estimated mean squared error is 13.8, is close to our estimate of 8.9

In [ ]:
# b) Create standalone model on entire training dataset
# c) Save model for later use

# SUMMARY
In this chapter you worked through a regression predictive modeling machine learning problem
from end-to-end using Python. Specifically, the steps covered were:

     Problem Definition (Boston house price data).
     Loading the Dataset.
     Analyze Data (some skewed distributions and correlated attributes).
     Evaluate Algorithms (Linear Regression looked good).
     Evaluate Algorithms with Standardization (KNN looked good).
     Algorithm Tuning (K=3 for KNN was best).
     Ensemble Methods (Bagging and Boosting, Extra Trees looked good).
     Tuning Ensemble Methods (getting the most from Extra Trees).
     Finalize Model (use all training data and confirm using validation dataset).